In [23]:
import time, pickle, ast, os
from tqdm import tqdm
import pandas as pd
import numpy as np

#script for reading/writing trees
from scripts.script import readP, writeTree
#script for build DTK
from scripts.script import createDTK
#script for parse sentences
from scripts.script import parse

from os import getcwd

import logging

logger = logging.getLogger(__name__)

print(getcwd());

E:\Tesi magistrale\KermitSimptc


In [24]:
# Download agnews
# !wget wget https://data.deepai.org/agnews.zip
#
# !unzip agnews.zip
dataset_name= "agnews"

In [25]:
tree = "(S)"
treeException = createDTK(tree)

In [26]:
from simptc import configs
def encode_sentence(prompted_text):
    try:
        tree = (parse(prompted_text))
        treeDTK = createDTK(tree)
    except Exception:
        tree, treeDTK = "(S)", treeException
    return treeDTK

def load_label_names():
    label_name_file = f"./agnews/label_names.txt"
    labels = []
    with open(label_name_file, "r") as f:
        for line in f:
            words = line.split("\n")[0].split(",")
            labels.append(words)
    return labels

def encode_label_names(label_name_file, model_name):
    print("encoding class anchor sentences...")
    embedding_dir = f'./agnews/kermit/embeddings'
    if not os.path.exists(embedding_dir):
        os.mkdir(embedding_dir)
    simcse = False
    labels = []
    with open(label_name_file, "r") as f:
        for line in f:
            words = line.split("\n")[0].split(",")
            labels.append(words)

    template_number = len(configs.TEMPLATES["agnews"])
    for template_id in range(template_number):
        model_name_with_prefix = model_name
        label_names_name = label_name_file.split('/')[-1][12:-4]   # extract label names name
        prompt_path = os.path.join(embedding_dir, f"prompt_{template_id}_{label_names_name}_{model_name_with_prefix}.npz")
        template = configs.TEMPLATES["agnews"][template_id]
        # create prompt embeddings:
        if not os.path.exists(prompt_path):
            print(f"generating prompt embeddings using {model_name} from {label_name_file}, template: {template}")
            print(f"embedding path = {prompt_path}")
            label_list = []
            prompt_embeddings = []
            for label_id, label_words in enumerate(labels):
                show_example = True
                for label_word in label_words:
                    prompted_text = template.replace("<mask>", label_word)
                    if show_example:
                        show_example = False
                        print(f"prompt exampe: {prompted_text}")
                    prompt_embeddings.append(encode_sentence(prompted_text))
                    label_list.append(label_id)
            prompt_embeddings = np.array(prompt_embeddings)
            label_list = np.array(label_list)
            print(f"embedding shape: {prompt_embeddings.shape}")
            np.savez(prompt_path, embedding_list=prompt_embeddings, label_list=label_list)
        else:
            print(f"prompt embeddings already exists at {prompt_path}")



In [27]:
def load_and_encode_agnews(data_path):
    label_list = []
    embedding_list = []
    with open(data_path, encoding="utf-8") as csv_file:
        split = pd.read_csv(data_path).sample(100)
        for row in tqdm(split.itertuples()):
            label, title, description = row._1, row.Title, row.Description
            label = int(label) - 1
            text = " ".join((title, description))
            label_list.append(label)
            sent_embedding = encode_sentence(text)
            embedding_list.append(sent_embedding)
    return embedding_list, label_list


def encode_and_save_sentences(model_name, split):
    logger.info("encoding train and test set...")
    # label_path = os.path.join(f"./datasets/TextClassification/{dataset}", f"{split}_labels.{suffix}")
    embedding_dir = os.path.join(f"./agnews/kermit/", "embeddings")

    if not os.path.exists(embedding_dir):
        os.mkdir(embedding_dir)
    path = os.path.join(embedding_dir, f"{split}_{model_name}.npz")

    # create testing embeddings:
    embedding_path = path
    # data_path = os.path.join(f"./datasets/TextClassification/{dataset}", f"{split}.{suffix}")
    if not os.path.exists(embedding_path):
        logger.info(f"generating train/test embeddings using {model_name} from kermit")
        embedding_list, label_list = load_and_encode_agnews(f"agnews/{split}.csv")
        label_list = np.array(label_list)
        embedding_list = np.array(embedding_list)
        logger.info(f"embedding size: {embedding_list.shape}, label shape: {label_list.shape}")
        logger.info(f"embeddings saved at {embedding_path}")
        np.savez(embedding_path, embedding_list=embedding_list, label_list=label_list)
    else:
        logger.info(f"embeddings already exists at {embedding_path}")
    return path


In [28]:
encode_label_names("./agnews/label_names_conceptnet_1000.txt", "Kermit")

encoding class anchor sentences...
generating prompt embeddings using Kermit from ./agnews/label_names_conceptnet_1000.txt, template: The news is about <mask>.
embedding path = ./agnews/kermit/embeddings\prompt_0_conceptnet_1000_Kermit.npz
prompt exampe: The news is about politicianess.
prompt exampe: The news is about fine play.
prompt exampe: The news is about high tech manufacturer.
prompt exampe: The news is about technology transfer.
embedding shape: (8, 4000)
generating prompt embeddings using Kermit from ./agnews/label_names_conceptnet_1000.txt, template: The news is related to <mask>.
embedding path = ./agnews/kermit/embeddings\prompt_1_conceptnet_1000_Kermit.npz
prompt exampe: The news is related to politicianess.
prompt exampe: The news is related to fine play.
prompt exampe: The news is related to high tech manufacturer.
prompt exampe: The news is related to technology transfer.
embedding shape: (8, 4000)
generating prompt embeddings using Kermit from ./agnews/label_names_co

In [29]:
encode_and_save_sentences("Kermit", "train")

100it [00:37,  2.67it/s]


'./agnews/kermit/embeddings\\train_Kermit.npz'

In [30]:
encode_and_save_sentences("Kermit", "test")

100it [00:34,  2.91it/s]


'./agnews/kermit/embeddings\\test_Kermit.npz'